In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


df=pd.read_csv(r"/home/natty/Videos/archive(6)/Amazon Sale Report.csv")
df.head(10)


/tmp/ipykernel_31304/985918885.py:10: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(r"/home/natty/Videos/archive(6)/Amazon Sale Report.csv")


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN
5,5,404-1490984-4578765,04-30-22,Shipped,Amazon,Amazon.in,Expedited,SET264,SET264-KR-NP-XL,Set,...,INR,824.00,GHAZIABAD,UTTAR PRADESH,201102.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN
6,6,408-5748499-6859555,04-30-22,Shipped,Amazon,Amazon.in,Expedited,J0095,J0095-SET-L,Set,...,INR,653.00,CHANDIGARH,CHANDIGARH,160036.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN
7,7,406-7807733-3785945,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3405,JNE3405-KR-S,kurta,...,INR,399.00,HYDERABAD,TELANGANA,500032.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
8,8,407-5443024-5233168,04-30-22,Cancelled,Amazon,Amazon.in,Expedited,SET200,SET200-KR-NP-A-XXXL,Set,...,NaN,NaN,HYDERABAD,TELANGANA,500008.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,NaN
9,9,402-4393761-0311520,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3461,JNE3461-KR-XXL,kurta,...,INR,363.00,Chennai,TAMIL NADU,600041.0,IN,NaN,False,NaN,NaN


In [2]:
df.shape


(128975, 24)

## cleaning data

In [3]:
df.isnull().sum()

index                     0
Order ID                  0
Date                      0
Status                    0
Fulfilment                0
Sales Channel             0
ship-service-level        0
Style                     0
SKU                       0
Category                  0
Size                      0
ASIN                      0
Courier Status         6872
Qty                       0
currency               7795
Amount                 7795
ship-city                33
ship-state               33
ship-postal-code         33
ship-country             33
promotion-ids         49153
B2B                       0
fulfilled-by          89698
Unnamed: 22           49050
dtype: int64

In [4]:
#since the column is unnamed and does not have a clear description or purposen so lets drop it 
df.drop(columns=['Unnamed: 22'], inplace=True)

In [5]:
# STANDARDIZING COLUMN NAMES
df.columns = [col.strip().replace("-", "_").lower() for col in df.columns]


In [6]:
#see if theres null data 
df.isnull().sum()

index                     0
order id                  0
date                      0
status                    0
fulfilment                0
sales channel             0
ship_service_level        0
style                     0
sku                       0
category                  0
size                      0
asin                      0
courier status         6872
qty                       0
currency               7795
amount                 7795
ship_city                33
ship_state               33
ship_postal_code         33
ship_country             33
promotion_ids         49153
b2b                       0
fulfilled_by          89698
dtype: int64

In [7]:
#had to check 1 by one 
df['courier status'].value_counts(dropna=False)

courier status
Shipped      109487
NaN            6872
Unshipped      6681
Cancelled      5935
Name: count, dtype: int64

## ** had this idea  most of the  time when its imazon.in the currency will be INR**

In [8]:

df.groupby('sales channel')['currency'].value_counts(dropna=False)
# i can impute the nan by INR

sales channel  currency
Amazon.in      INR         121180
               NaN           7671
Non-Amazon     NaN            124
Name: count, dtype: int64

In [9]:


# Imputing missing 'currency' for 'Amazon.in' sales channel with 'INR'
df.loc[(df['sales channel'] == 'Amazon.in') & (df['currency'].isnull()), 'currency'] = 'INR'
# For the 'Non-Amazon' sales channel, we mark the currency as 'Unknown'
df.loc[(df['sales channel'] == 'Non-Amazon') & (df['currency'].isnull()), 'currency'] = 'Unknown'
print(df.groupby('sales channel')
['currency'].value_counts(dropna=False))



sales channel  currency
Amazon.in      INR         128851
Non-Amazon     Unknown        124
Name: count, dtype: int64


In [10]:
df['promotion_ids'].value_counts(dropna=False)

promotion_ids
NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [11]:
# Looks like 'promotion_ids' means there is no promotion applied in this dataframe, let's fill NaN with 'No Promotion' 
df['promotion_ids'].fillna('No Promotion', inplace=True)

/tmp/ipykernel_31304/365740346.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['promotion_ids'].fillna('No Promotion', inplace=True)


In [12]:
df['fulfilled_by'].value_counts(dropna=False)

fulfilled_by
NaN          89698
Easy Ship    39277
Name: count, dtype: int64

In [13]:
# We can say that if NaN are not 'Easy Ship' we can fiil it with normal ship
df['fulfilled_by'].fillna('Normal Ship', inplace=True)

/tmp/ipykernel_31304/703851686.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['fulfilled_by'].fillna('Normal Ship', inplace=True)


In [14]:
df.isnull().sum()

index                    0
order id                 0
date                     0
status                   0
fulfilment               0
sales channel            0
ship_service_level       0
style                    0
sku                      0
category                 0
size                     0
asin                     0
courier status        6872
qty                      0
currency                 0
amount                7795
ship_city               33
ship_state              33
ship_postal_code        33
ship_country            33
promotion_ids            0
b2b                      0
fulfilled_by             0
dtype: int64

In [15]:

#replacing null by uknown 
df['courier status'].fillna('Unknown', inplace=True)

/tmp/ipykernel_31304/4266164407.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['courier status'].fillna('Unknown', inplace=True)


## there's a 'null' file in 'amount' column so i took look at it in Excel and i found out everytime when 'courier status' is 'canceled' or 'Unknown'(i renamed null files with uknown) the amount will be 'null' so i have to check it 

In [29]:

# Check for null values in the 'amount' column
null_amounts = df[df['amount'].isnull()]

# Verify the 'courier status' for rows with null 'amount'
null_amount_status = null_amounts['courier status']

# Print unique statuses for null 'amount'
print("Unique 'courier status' values for null 'amount':", null_amount_status.unique())

# Filter rows where 'courier status' is 'Cancelled' or 'Unknown'
relevant_statuses = ['Cancelled', 'Unknown']
relevant_null_amounts = null_amounts[null_amounts['courier status'].isin(relevant_statuses)]

# Count and compare occurrences
total_null_amounts = len(null_amounts)
relevant_null_amounts_count = len(relevant_null_amounts)
percentage_relevant = (relevant_null_amounts_count / total_null_amounts) * 100 if total_null_amounts != 0 else 0

# Print results
print(f"Total null 'amount' values: {total_null_amounts}")
print(f"Null 'amount' values with 'courier status' as 'Cancelled' or 'Unknown': {relevant_null_amounts_count}")
print(f"Percentage of null 'amount' values with 'courier status' as 'Cancelled' or 'Unknown': {percentage_relevant:.2f}%")

# Additionally, check if all 'Cancelled' or 'Unknown' statuses have null 'amount'
status_is_relevant = df[df['courier status'].isin(relevant_statuses)]
status_is_relevant_null_amount = status_is_relevant[status_is_relevant['amount'].isnull()]
total_relevant_status = len(status_is_relevant)
relevant_status_null_amount = len(status_is_relevant_null_amount)
percentage_status_null = (relevant_status_null_amount / total_relevant_status) * 100 if total_relevant_status != 0 else 0

# Print results
print(f"Total 'courier status' as 'Cancelled' or 'Unknown': {total_relevant_status}")
print(f"'Cancelled' or 'Unknown' statuses with null 'amount': {relevant_status_null_amount}")
print(f"Percentage of 'Cancelled' or 'Unknown' statuses with null 'amount': {percentage_status_null:.2f}%")



Unique 'courier status' values for null 'amount': ['Cancelled' 'Unknown' 'Unshipped']
Total null 'amount' values: 7795
Null 'amount' values with 'courier status' as 'Cancelled' or 'Unknown': 7671
Percentage of null 'amount' values with 'courier status' as 'Cancelled' or 'Unknown': 98.41%
Total 'courier status' as 'Cancelled' or 'Unknown': 12807
'Cancelled' or 'Unknown' statuses with null 'amount': 7671
Percentage of 'Cancelled' or 'Unknown' statuses with null 'amount': 59.90%
